Vehicle Routing Problem:
- Install Google OR Tools package by uncommenting the first cell and executing it.

In [2]:
# !pip install ortools

  Using cached ortools-8.2.8710-cp38-cp38-win_amd64.whl (42.3 MB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached protobuf-3.15.8-py2.py3-none-any.whl (173 kB)


In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [82]:
# function for creating dummy distance matrix for warehouse and demand points
def create_data_model(warehouse_no: int):
    # Data matrices for 3 warehouses
    w1 = [[0, 6, 5, 5, 6, 8, 10, 4, 2, 5, 9, 9, 5, 5, 7, 8, 4, 2, 9, 7, 4],
        [10, 0, 6, 4, 1, 7, 4, 4, 3, 2, 7, 7, 7, 5, 6, 10, 8, 7, 5, 8, 10],
        [8, 3, 0, 1, 8, 2, 8, 5, 1, 1, 1, 1, 5, 6, 10, 6, 5, 1, 4, 7, 10],
        [10, 6, 3, 0, 1, 5, 7, 6, 6, 1, 6, 6, 7, 5, 10, 9, 5, 5, 7, 8, 1],
        [7, 10, 1, 7, 0, 3, 4, 5, 6, 9, 8, 5, 2, 4, 7, 4, 2, 3, 10, 5, 9],
        [8, 2, 10, 5, 8, 0, 6, 3, 7, 2, 9, 3, 5, 2, 1, 5, 9, 3, 5, 1, 3],
        [1, 6, 7, 6, 3, 4, 0, 5, 1, 6, 4, 9, 10, 2, 8, 6, 8, 6, 8, 4, 5],
        [5, 6, 10, 3, 7, 5, 1, 0, 4, 7, 3, 6, 9, 2, 8, 1, 3, 10, 9, 2, 6],
        [1, 7, 3, 9, 4, 5, 4, 4, 0, 2, 4, 9, 7, 9, 3, 10, 5, 10, 3, 9, 4],
        [5, 6, 6, 8, 8, 4, 1, 1, 6, 0, 7, 10, 1, 10, 4, 8, 1, 6, 6, 8, 1],
        [6, 6, 1, 3, 9, 7, 5, 7, 10, 1, 0, 9, 2, 9, 9, 9, 8, 3, 9, 9, 9],
        [8, 7, 3, 8, 5, 6, 8, 1, 3, 9, 4, 0, 7, 2, 5, 10, 4, 6, 6, 6, 3],
        [6, 2, 9, 6, 10, 6, 6, 2, 7, 2, 1, 10, 0, 4, 8, 6, 3, 5, 1, 2, 6],
        [6, 3, 1, 9, 8, 2, 10, 1, 4, 9, 8, 9, 2, 0, 4, 1, 6, 1, 3, 3, 4],
        [5, 5, 8, 3, 8, 6, 10, 1, 7, 10, 5, 9, 7, 4, 0, 2, 5, 9, 9, 7, 8],
        [8, 10, 1, 2, 10, 2, 3, 5, 8, 7, 6, 1, 7, 7, 7, 0, 10, 10, 9, 1, 9],
        [8, 6, 5, 10, 5, 6, 7, 1, 3, 10, 9, 9, 8, 8, 9, 3, 0, 6, 7, 5, 4],
        [6, 8, 7, 10, 1, 10, 2, 6, 10, 6, 9, 9, 10, 3, 9, 7, 4, 0, 2, 7, 5],
        [7, 4, 7, 6, 5, 7, 2, 7, 4, 9, 3, 8, 4, 10, 10, 4, 6, 10, 0, 6, 10],
        [4, 4, 3, 6, 3, 3, 1, 10, 8, 5, 5, 1, 5, 10, 9, 7, 1, 1, 10, 0, 2],
        [9, 2, 6, 8, 9, 8, 5, 2, 2, 3, 1, 3, 3, 6, 2, 2, 2, 5, 7, 9, 0]
        ]
    
    w2 = [
        [0, 6, 10, 7, 2, 1, 8, 7, 9, 2, 6, 3, 6, 8, 9, 3, 2, 5, 6, 3, 4, 9, 5, 10, 6],
        [4, 0, 10, 6, 2, 10, 10, 3, 1, 9, 6, 2, 10, 5, 4, 7, 10, 8, 9, 4, 5, 10, 4, 9, 4],
        [10, 7, 0, 2, 1, 7, 1, 8, 9, 1, 3, 4, 2, 5, 2, 9, 4, 5, 3, 4, 1, 10, 6, 1, 4],
        [8, 6, 7, 0, 4, 4, 3, 10, 7, 5, 6, 6, 2, 2, 4, 7, 4, 5, 4, 2, 9, 1, 9, 3, 7],
        [1, 10, 7, 6, 0, 9, 2, 6, 4, 2, 7, 9, 7, 4, 5, 7, 10, 1, 9, 8, 2, 6, 8, 9, 3],
        [1, 1, 7, 2, 8, 0, 3, 10, 7, 4, 5, 7, 9, 4, 3, 5, 4, 3, 1, 2, 4, 1, 1, 9, 7],
        [10, 6, 9, 1, 5, 3, 0, 2, 3, 8, 9, 3, 6, 2, 2, 2, 1, 10, 6, 4, 7, 9, 3, 2, 9],
        [2, 8, 10, 5, 4, 2, 3, 0, 8, 4, 9, 5, 1, 4, 6, 1, 4, 9, 3, 7, 9, 5, 3, 7, 9],
        [3, 1, 5, 10, 6, 9, 6, 4, 0, 4, 6, 8, 8, 4, 7, 3, 3, 10, 7, 5, 7, 1, 6, 5, 3],
        [6, 6, 3, 5, 3, 7, 6, 2, 8, 0, 7, 1, 4, 6, 10, 1, 2, 8, 8, 5, 2, 4, 6, 5, 4],
        [7, 6, 5, 4, 4, 4, 6, 7, 6, 8, 0, 6, 8, 3, 8, 5, 5, 4, 4, 6, 9, 5, 6, 10, 1],
        [10, 6, 7, 6, 8, 7, 8, 3, 9, 2, 10, 0, 7, 8, 3, 9, 1, 3, 2, 6, 4, 4, 3, 1, 3],
        [1, 3, 1, 1, 3, 1, 3, 10, 6, 7, 3, 5, 0, 6, 2, 9, 2, 8, 2, 4, 2, 7, 4, 9, 4],
        [10, 3, 10, 1, 6, 1, 9, 3, 9, 6, 7, 2, 1, 0, 7, 2, 2, 2, 5, 9, 5, 5, 3, 10, 3],
        [3, 7, 8, 4, 8, 7, 4, 2, 6, 4, 10, 1, 3, 9, 0, 1, 10, 8, 2, 9, 10, 2, 3, 1, 7],
        [7, 9, 5, 4, 10, 1, 1, 4, 7, 2, 4, 8, 6, 9, 3, 0, 10, 7, 10, 5, 3, 4, 7, 2, 9],
        [6, 3, 6, 7, 2, 9, 7, 9, 6, 4, 1, 7, 9, 10, 6, 9, 0, 9, 7, 4, 7, 10, 10, 1, 10],
        [1, 7, 9, 10, 7, 8, 5, 1, 10, 1, 2, 4, 5, 3, 4, 10, 7, 0, 8, 2, 6, 1, 5, 6, 1],
        [9, 2, 10, 1, 9, 1, 8, 4, 4, 8, 8, 4, 8, 6, 7, 3, 2, 8, 0, 9, 2, 7, 6, 9, 8],
        [7, 7, 4, 2, 6, 1, 5, 1, 1, 2, 7, 3, 6, 8, 3, 7, 10, 9, 5, 0, 9, 1, 1, 3, 3],
        [6, 6, 3, 7, 10, 7, 6, 2, 6, 4, 6, 8, 7, 4, 2, 5, 10, 4, 1, 4, 0, 7, 9, 2, 6],
        [1, 6, 10, 5, 6, 7, 4, 8, 10, 6, 7, 8, 6, 7, 7, 3, 9, 3, 4, 3, 9, 0, 2, 1, 7],
        [10, 7, 5, 8, 10, 2, 7, 9, 8, 1, 5, 2, 2, 5, 4, 9, 5, 2, 3, 6, 9, 7, 0, 6, 8],
        [4, 10, 1, 7, 4, 3, 9, 5, 9, 7, 5, 8, 8, 8, 6, 7, 3, 7, 3, 3, 9, 6, 10, 0, 3],
        [8, 6, 8, 3, 1, 10, 10, 3, 2, 6, 1, 6, 9, 4, 10, 3, 7, 10, 5, 8, 6, 10, 6, 7, 0]

    ]
    
    w3 = [[0, 7, 1, 3, 1, 9, 2, 9, 2, 5, 8, 7, 7, 8, 5, 7, 6, 3, 8, 4, 5],
        [9, 0, 1, 3, 3, 6, 1, 4, 3, 4, 3, 3, 7, 1, 5, 3, 9, 8, 7, 6, 2],
        [3, 8, 0, 6, 9, 1, 4, 4, 7, 7, 8, 9, 1, 7, 1, 5, 1, 3, 2, 2, 2],
        [7, 6, 1, 0, 1, 1, 9, 1, 5, 2, 6, 4, 9, 7, 9, 3, 3, 7, 8, 7, 1],
        [7, 7, 6, 9, 0, 1, 7, 7, 2, 5, 6, 1, 3, 9, 8, 7, 5, 5, 8, 8, 8],
        [8, 3, 1, 3, 8, 0, 1, 3, 3, 3, 2, 3, 8, 1, 7, 6, 6, 9, 5, 5, 2],
        [9, 6, 7, 1, 2, 4, 0, 4, 3, 4, 9, 8, 9, 6, 1, 8, 4, 7, 7, 1, 4],
        [8, 4, 6, 1, 6, 3, 5, 0, 5, 6, 4, 9, 6, 3, 8, 1, 8, 5, 4, 5, 3],
        [4, 2, 3, 3, 4, 9, 8, 8, 0, 2, 6, 6, 6, 2, 5, 5, 6, 4, 7, 3, 6],
        [6, 9, 8, 5, 8, 1, 7, 3, 9, 0, 3, 4, 4, 1, 9, 8, 1, 4, 5, 4, 2],
        [8, 8, 9, 4, 4, 7, 2, 7, 3, 5, 0, 1, 8, 9, 5, 7, 2, 3, 5, 3, 1],
        [2, 7, 6, 6, 5, 2, 5, 3, 6, 8, 7, 0, 6, 6, 3, 8, 4, 1, 8, 1, 9],
        [5, 9, 8, 6, 7, 3, 8, 5, 5, 6, 4, 3, 0, 3, 4, 8, 9, 3, 8, 7, 4],
        [1, 4, 2, 3, 5, 6, 3, 8, 6, 4, 9, 9, 1, 0, 6, 2, 9, 3, 2, 2, 3],
        [2, 7, 5, 8, 1, 7, 9, 6, 6, 8, 9, 6, 3, 8, 0, 9, 6, 6, 5, 2, 8],
        [7, 7, 5, 9, 6, 7, 5, 8, 1, 9, 4, 6, 4, 8, 7, 0, 6, 3, 4, 1, 9],
        [8, 6, 9, 8, 1, 2, 8, 2, 1, 8, 7, 3, 6, 6, 2, 9, 0, 6, 2, 5, 1],
        [4, 3, 8, 4, 6, 9, 5, 2, 9, 5, 1, 6, 5, 8, 3, 1, 9, 0, 2, 2, 9],
        [6, 8, 3, 9, 1, 8, 7, 1, 6, 8, 9, 7, 3, 3, 6, 3, 7, 9, 0, 2, 8],
        [9, 6, 7, 9, 9, 5, 3, 2, 8, 2, 6, 2, 6, 8, 1, 7, 6, 9, 6, 0, 8],
        [3, 4, 4, 5, 4, 6, 2, 2, 2, 6, 8, 1, 5, 3, 1, 6, 8, 5, 5, 5, 0]

    ]
    warehouse_list = [w1,w2,w3]
    data = {}
    data['distance_matrix'] = warehouse_list[warehouse_no-1]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [76]:
# function for printing the shortes routes taken by each vehicle for each warehouse
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        total_distance += route_distance
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    print('Total distance: {}m'.format(total_distance))

In [77]:
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [78]:
# function for solving the vehicle routing problem
"""Solve the CVRP problem."""
def get_optimal_routes(warehouse_no: int):
    # Instantiate the data problem.
    data = create_data_model(warehouse_no)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

## Obtaining shortest routes for all vehicles for each warehouse.

In [79]:
get_optimal_routes(1)

Route for vehicle 0:
 0 ->  5 ->  1 ->  8 ->  16 ->  10 -> 0
Distance of the route: 8m

Route for vehicle 1:
 0 ->  19 ->  7 ->  15 ->  6 ->  13 ->  12 -> 0
Distance of the route: 10m

Route for vehicle 2:
 0 ->  9 ->  11 ->  14 -> 0
Distance of the route: 8m

Route for vehicle 3:
 0 ->  2 ->  20 ->  18 ->  3 ->  4 ->  17 -> 0
Distance of the route: 10m

Maximum of the route distances: 10m
Total distance: 36m


In [80]:
get_optimal_routes(2)

Route for vehicle 0:
 0 ->  9 ->  11 ->  23 ->  2 ->  6 ->  14 ->  7 ->  15 ->  5 -> 0
Distance of the route: 13m

Route for vehicle 1:
 0 -> 0
Distance of the route: 0m

Route for vehicle 2:
 0 ->  19 ->  8 ->  1 ->  22 ->  17 ->  21 -> 0
Distance of the route: 13m

Route for vehicle 3:
 0 ->  16 ->  10 ->  24 ->  4 ->  20 ->  18 ->  3 ->  13 ->  12 -> 0
Distance of the route: 13m

Maximum of the route distances: 13m
Total distance: 39m


In [81]:
get_optimal_routes(3)

Route for vehicle 0:
 0 ->  5 ->  1 ->  8 ->  16 ->  10 -> 0
Distance of the route: 8m

Route for vehicle 1:
 0 ->  19 ->  7 ->  15 ->  6 ->  13 ->  12 -> 0
Distance of the route: 10m

Route for vehicle 2:
 0 ->  9 ->  11 ->  14 -> 0
Distance of the route: 8m

Route for vehicle 3:
 0 ->  2 ->  20 ->  18 ->  3 ->  4 ->  17 -> 0
Distance of the route: 10m

Maximum of the route distances: 10m
Total distance: 36m
